# cuGraph Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

### Imports

In [1]:
import cudf
import cugraph
import json 
import pandas as pd

### Create graph dataframes

We use the Star Wars social network graph prepared by Evelina Gabasova: Gabasova, E. (2016). Star Wars social network. DOI: https://doi.org/10.5281/zenodo.1411479.

In [2]:
interactions = json.loads(open('data/starwars-full-interactions.json', 'r').read())

edges = cudf.from_pandas(pd.DataFrame(interactions['links']))
nodes = cudf.from_pandas(pd.DataFrame(interactions['nodes'])).drop('colour', axis=1)

In [3]:
edges.head()

,source,target,value
0,1,0,1
1,2,3,1
2,4,3,1
3,0,3,1
4,4,0,26


In [4]:
nodes.head()

,name,value
0,QUI-GON,61
1,NUTE GUNRAY,24
2,PK-4,3
3,TC-14,4
4,OBI-WAN,147


---

# Graph creation

---

#### cugraph.structure.graph.Graph.from_cudf_edgelist()

In [5]:
edges_cudf = cudf.DataFrame([
    (0,10)
    ,(0,11)
    ,(0,12)
    ,(1,10)
    ,(1,12)
], columns=['src', 'dst'])

nodes_cudf = [None, None]
nodes[0] = cudf.Series([0,1])
nodes[1] = cudf.Series([10,11,12])

g = cugraph.Graph()

g.add_nodes_from(nodes_cudf[0], bipartite=True)
g.add_nodes_from(nodes_cudf[1], bipartite=True)

g.from_cudf_edgelist(edges_cudf, source='src', destination='dst', edge_attr=None)

#### cugraph.structure.graph.Graph.from_cudf_adjlist()

In [6]:
import numpy as np
from scipy.sparse import coo_matrix

e_df = edges.to_pandas()

adj_list = coo_matrix(
    (
        e_df['value']
        , (e_df['source'], e_df['target'])
    )
).tocsr()

g = cugraph.Graph()
g.from_cudf_adjlist(cudf.Series(adj_list.indptr), cudf.Series(adj_list.indices))

#### cugraph.structure.graph.Graph.from_cudf_edgelist()

In [7]:
g = cugraph.Graph()
g.from_cudf_edgelist(
    edges
    , source='source'
    , destination='target'
    , edge_attr='value'
    , renumber=False
)